In [13]:
from preprocessing import * 
from sklearn import svm
from sklearn import pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

from sklearn.multiclass import OneVsRestClassifier

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Extract data samples from raw GDF files

In [16]:
# convert_data()
convert_file("data/A01T.gdf")

Extracting EDF parameters from d:\Study\GP\EEG-Motor-Imagery-classifier\data\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...


c:\Users\Mohamad Abo Bakr\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\io\edf\edf.py:1123: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
c:\Users\Mohamad Abo Bakr\AppData\Local\Programs\Python\Python39\lib\contextlib.py:124: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
[[     0      0      5]
 [     0      0      3]
 [ 29683      0      5]
 ...
 [670550      0      6]
 [670550      0      1]
 [671050      0      7]] {'1023': 1, '1072': 2, '276': 3, '277': 4, '32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10}
[0 0 5]
[0 0 3]
[29683     0     5]
[29683     0     4]
[49955     0     5]
[49955     0     2]
[91518     0     5]
[91868     0     6]
[92368     0    10]
[93871     0     6]
[94371     0     9]
[95789     0     6]
[96289     0     8]
[97741     0     6]
[98241     0     7]
[99749     0     6]
[100249      0      7]
[101860      0      6]
[102360      0      8]
[103777      0      6]
[104277      0      9]
[105827      0      6]
[106327      0     10]
[107758      0      6]
[108258      0      8]
[109756      0      6]
[110256      0      9]
[111662      0      6]
[112162      0      7]
[113557      0      6]
[114057      0      

# Read and split the data

In [8]:
i = 8

x, y = read_file(i)

# X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 100, test_size = 0.2)

# print("final shapes are: ", X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(x.shape)


(288, 25, 313)


# Extract features from test and train data

In [10]:
# train_coeff = featurize(X_train)
# coeff_len = len(train_coeff)

# csp = [mne.decoding.CSP(8) for _ in range(coeff_len)]
# X_train_f = np.concatenate(tuple(csp[x].fit_transform(train_coeff[x], y_train) for x  in range(coeff_len)),axis=-1)
train_coeff = featurize(x)
coeff_len = len(train_coeff)

csp = [mne.decoding.CSP(8) for _ in range(coeff_len)]
X_train_f = np.concatenate(tuple(csp[x].fit_transform(train_coeff[x], y) for x  in range(coeff_len)),axis=-1)

# test_coeff = featurize(X_test)
# X_test_f = np.concatenate(tuple(csp[x].transform(test_coeff[x]) for x  in range(coeff_len)),axis=-1)
  

Computing rank from data with rank=None
    Using tolerance 66 (2.2e-16 eps * 25 dim * 1.2e+16  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 65 (2.2e-16 eps * 25 dim * 1.2e+16  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 69 (2.2e-16 eps * 25 dim * 1.2e+16  max singular value)


c:\Users\Mohamad Abo Bakr\AppData\Local\Programs\Python\Python39\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 7 is too high: all coefficients will experience boundary effects.
  warnings.warn(


    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 60 (2.2e-16 eps * 25 dim * 1.1e+16  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 27 (2.2e-16 eps * 25 dim * 4.9e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 27 (2.2e-16 eps * 25 dim * 4.8e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimati

# Train model and test

In [11]:
clf = pipeline.make_pipeline(StandardScaler(), svm.SVC())
clf.fit(X_train_f, y)

print("Accuracy is ", sum(clf.predict(X_train_f) == y) / len(x))
print(len(x))
# clf = pipeline.make_pipeline(StandardScaler(), svm.SVC())
# clf.fit(X_train_f, y_train)

# print("Accuracy is ", sum(clf.predict(X_train_f) == y_train) / len(X_train))
# print(len(X_train))

# print("test Accuracy is ", sum(clf.predict(X_test_f) == y_test) / len(X_test))
# print(len(X_test))

print("kappa score on train is: ", cohen_kappa_score(clf.predict(X_train_f), y))
# print("kappa score on test is: ", cohen_kappa_score(clf.predict(X_test_f), y_test))


Accuracy is  0.9861111111111112
288
kappa score on train is:  0.9814814814814815


# Save the trained model

In [12]:
dump(clf, "model.joblib")
dump(csp, "csp.joblib")

['csp.joblib']